# Thực hành 2 : Cách thực thi song song trên CUDA
**Thông tin sinh viên** :

Hoàng Minh Thanh (18424062)

Jupyter notebook (Online) : https://colab.research.google.com/drive/1piW2XzDPXJOg3WBliJAvCGBbQqDXeT8N

Thực hiện chạy trên Google Colab

Cài đặt plugin chạy GPU trên Google Colab

In [2]:
%%bash
nvcc --version
rm -rf /content/*
pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-p7ngbeia
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=70c439a373062d30599a3f36e709cb93b8deef5d52b6bb7307c8910a7e7358e0
  Stored in directory: /tmp/pip-ephem-wheel-cache-ga32o48y/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-p7ngbeia


In [3]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


Tải github private repository chứa file in.pnm cần lấy và các file khác

Vì đây là private repository nên phải cấu hình clone bằng ssh

Tham khảo tại : https://towardsdatascience.com/using-git-with-colab-via-ssh-175e8f3751ec

In [8]:
import os
os.chdir("/content/")
from google.colab import files
uploaded = files.upload()
!ls

Saving ssh.tar.gz to ssh.tar.gz
src  ssh.tar.gz


In [9]:
%%bash
rm -rf /root/.ssh
mkdir /root/.ssh

tar xvzf ssh.tar.gz
cp ssh-colab/* /root/.ssh && rm -rf ssh-colab && rm -rf ssh.tar.gz
chmod 700 /root/.ssh

touch /root/.ssh/known_hosts
ssh-keyscan github.com >> /root/.ssh/known_hosts
chmod 644 /root/.ssh/known_hosts
chmod 400 /root/.ssh/id_rsa_thanh

ssh-colab/
ssh-colab/config
ssh-colab/id_rsa_thanh
ssh-colab/id_rsa_thanh.pub


# github.com:22 SSH-2.0-babeld-97a5c183
# github.com:22 SSH-2.0-babeld-97a5c183
# github.com:22 SSH-2.0-babeld-97a5c183


In [16]:
%%bash
rm -rf /content/*
git config --global user.email "hmthanhgm@gmail.com"
git config --global user.name "hmthanh"

Di chuyển các file cần thiết ra /content/

In [17]:
%%bash
ssh-agent /bin/bash
git clone git@github.com:hmthanh/paralell_programming.git
mv /content/paralell_programming/TH_2/* /content/
rm -rf /content/paralell_programming/

Cloning into 'paralell_programming'...


## Mã nguồn chương trình chính

In [19]:
%%cu
#include <stdio.h>

#define CHECK(call)                                                            \
{                                                                              \
    const cudaError_t error = call;                                            \
    if (error != cudaSuccess)                                                  \
    {                                                                          \
        fprintf(stderr, "Error: %s:%d, ", __FILE__, __LINE__);                 \
        fprintf(stderr, "code: %d, reason: %s\n", error,                       \
                cudaGetErrorString(error));                                    \
        exit(EXIT_FAILURE);                                                    \
    }                                                                          \
}

struct GpuTimer
{
    cudaEvent_t start;
    cudaEvent_t stop;

    GpuTimer()
    {
        cudaEventCreate(&start);
        cudaEventCreate(&stop);
    }

    ~GpuTimer()
    {
        cudaEventDestroy(start);
        cudaEventDestroy(stop);
    }

    void Start()
    {
        cudaEventRecord(start, 0);
    }

    void Stop()
    {
        cudaEventRecord(stop, 0);
    }

    float Elapsed()
    {
        float elapsed;
        cudaEventSynchronize(stop);
        cudaEventElapsedTime(&elapsed, start, stop);
        return elapsed;
    }
};

__global__ void reduceBlksKernel1(int * in, int n, int * out)
{
	// TODO
	int i = blockIdx.x * blockDim.x * 2 + threadIdx.x * 2;
    for (int stride = 1; stride < 2 * blockDim.x; stride *= 2)
    {
        if ((threadIdx.x % stride) == 0)
            if (i + stride < n)
                in[i] += in[i + stride];

        __syncthreads(); // Synchronize within each block
    }
    
    if (threadIdx.x == 0)
        out[blockIdx.x] = in[blockIdx.x * blockDim.x * 2];
}

__global__ void reduceBlksKernel2(int * in, int n, int * out)
{
	// TODO
	int numElemsBeforeBlk = blockIdx.x * blockDim.x * 2;
    
    for (int stride = 1; stride < 2 * blockDim.x; stride *= 2)
    {
        int i = numElemsBeforeBlk + threadIdx.x * 2 * stride;
        //int index = 2 * stride * tid;
        if (threadIdx.x < blockDim.x / stride){
            if (i + stride < n){
                in[i] += in[i + stride];
            }
        }

        __syncthreads(); // Synchronize within each block
    }

    if (threadIdx.x == 0)
        out[blockIdx.x] = in[numElemsBeforeBlk];
}

__global__ void reduceBlksKernel3(int * in, int n, int * out)
{
	// TODO
	int numElemsBeforeBlk = blockIdx.x * blockDim.x * 2;
    
    for (int stride = blockDim.x; stride > 0; stride >>= 1)
    {
        int i = numElemsBeforeBlk + threadIdx.x;
        if (threadIdx.x < stride)
            in[i] += in[i + stride];

        __syncthreads();
    }
    if (threadIdx.x == 0)
        out[blockIdx.x] = in[numElemsBeforeBlk];
}

int reduce(int const * in, int n,
			bool useDevice=false, dim3 blockSize=dim3(1), int kernelType=1)
{

	int result = 0; // Init
	if (useDevice == false)
	{
		result = in[0];
		for (int i = 1; i < n; i++)
			result += in[i];
	}
	else // Use device
	{
		// Allocate device memories
		int * d_in, * d_out;
		dim3 gridSize((n-1)/(blockSize.x*2) + 1); // TODO: Compute gridSize from n and blockSize
		CHECK(cudaMalloc(&d_in, n * sizeof(int)));
		CHECK(cudaMalloc(&d_out, gridSize.x * sizeof(int)));

		// Copy data to device memory
		CHECK(cudaMemcpy(d_in, in, n*sizeof(int), cudaMemcpyHostToDevice));

		// Call kernel
		GpuTimer timer;
		timer.Start();
		if (kernelType == 1)
			reduceBlksKernel1<<<gridSize, blockSize>>>(d_in, n, d_out);
		else if (kernelType == 2)
			reduceBlksKernel2<<<gridSize, blockSize>>>(d_in, n, d_out);
		else
			reduceBlksKernel3<<<gridSize, blockSize>>>(d_in, n, d_out);
		timer.Stop();
		float kernelTime = timer.Elapsed();
		cudaDeviceSynchronize();
		CHECK(cudaGetLastError());

		// Copy result from device memory
		int * out = (int *)malloc(gridSize.x * sizeof(int));
		CHECK(cudaMemcpy(out, d_out, gridSize.x*sizeof(int), cudaMemcpyDeviceToHost));

		// Free device memories
		CHECK(cudaFree(d_in));
		CHECK(cudaFree(d_out));

		// Host do the rest of the work
		timer.Start();
		result = out[0];
		for (int i = 1; i < gridSize.x; i++)
		{
			result += out[i];
		}
		timer.Stop();
		float postKernelTime = timer.Elapsed();

		// Free memory
		free(out);

		// Print info
		printf("\nKernel %d\n", kernelType);
		printf("Grid size: %d, block size: %d\n", gridSize.x, blockSize.x);
		printf("Kernel time = %f ms, post-kernel time = %f ms\n", kernelTime, postKernelTime);
	}

	return result;
}

void checkCorrectness(int r1, int r2)
{
	if (r1 == r2)
		printf("CORRECT :)\n");
	else
		printf("INCORRECT :(\n");
}

void printDeviceInfo()
{
	cudaDeviceProp devProv;
    CHECK(cudaGetDeviceProperties(&devProv, 0));
    printf("**********GPU info**********\n");
    printf("Name: %s\n", devProv.name);
    printf("Compute capability: %d.%d\n", devProv.major, devProv.minor);
    printf("Num SMs: %d\n", devProv.multiProcessorCount);
    printf("Max num threads per SM: %d\n", devProv.maxThreadsPerMultiProcessor); 
    printf("Max num warps per SM: %d\n", devProv.maxThreadsPerMultiProcessor / devProv.warpSize);
    printf("GMEM: %lu bytes\n", devProv.totalGlobalMem);
    printf("****************************\n\n");

}
int main(int argc, char ** argv)
{
	printDeviceInfo();

	// Set up input size
    int n = (1 << 24) + 1;
    printf("Input size: %d\n", n);

    // Set up input data
    int * in = (int *) malloc(n * sizeof(int));
    for (int i = 0; i < n; i++)
    {
        // Generate a random integer in [0, 255]
        in[i] = (int)(rand() & 0xFF);
    }

    // Reduce NOT using device
    int correctResult = reduce(in, n);

    // Reduce using device, kernel1
    dim3 blockSize(512); // Default
    if (argc == 2)
    	blockSize.x = atoi(argv[1]);
    int result1 = reduce(in, n, true, blockSize, 1);
    checkCorrectness(result1, correctResult);

    // Reduce using device, kernel2
    int result2 = reduce(in, n, true, blockSize, 2);
    checkCorrectness(result2, correctResult);

    // Reduce using device, kernel3
    int result3 = reduce(in, n, true, blockSize, 3);
    checkCorrectness(result3, correctResult);

    // Free memories
    free(in);
}

**********GPU info**********
Name: Tesla P100-PCIE-16GB
Compute capability: 6.0
Num SMs: 56
Max num threads per SM: 2048
Max num warps per SM: 64
GMEM: 17071734784 bytes
****************************

Input size: 16777217

Kernel 1
Grid size: 16385, block size: 512
Kernel time = 1.228320 ms, post-kernel time = 0.038944 ms
CORRECT :)

Kernel 2
Grid size: 16385, block size: 512
Kernel time = 1.085728 ms, post-kernel time = 0.039072 ms
CORRECT :)

Kernel 3
Grid size: 16385, block size: 512
Kernel time = 0.542656 ms, post-kernel time = 0.039616 ms
CORRECT :)



Biên dịch file bt2.cu

In [20]:
!nvcc bt2.cu -o bt2

# Chạy với các kích thước block khác nhau

* 1024

In [23]:
!./bt2 1024

**********GPU info**********
Name: Tesla P100-PCIE-16GB
Compute capability: 6.0
Num SMs: 56
Max num threads per SM: 2048
Max num warps per SM: 64
GMEM: 17071734784 bytes
****************************

Input size: 16777217

Kernel 1
Grid size: 8193, block size: 1024
Kernel time = 1.442144 ms, post-kernel time = 0.031968 ms
CORRECT :)

Kernel 2
Grid size: 8193, block size: 1024
Kernel time = 1.266368 ms, post-kernel time = 0.020128 ms
CORRECT :)

Kernel 3
Grid size: 8193, block size: 1024
Kernel time = 0.589472 ms, post-kernel time = 0.019424 ms
CORRECT :)


Kiểm tra thông tin phần cứng

In [24]:
%%cu
#include <cuda_runtime.h>
#include <stdio.h>

#define CHECK(call)                                                            \
{                                                                              \
    const cudaError_t error = call;                                            \
    if (error != cudaSuccess)                                                  \
    {                                                                          \
        fprintf(stderr, "Error: %s:%d, ", __FILE__, __LINE__);                 \
        fprintf(stderr, "code: %d, reason: %s\n", error,                       \
                cudaGetErrorString(error));                                    \
        exit(EXIT_FAILURE);                                                    \
    }                                                                          \
}

int main(int argc, char **argv)
{
    printf("%s Starting...\n", argv[0]);

    int deviceCount = 0;
    cudaGetDeviceCount(&deviceCount);

    if (deviceCount == 0)
    {
        printf("There are no available device(s) that support CUDA\n");
    }
    else
    {
        printf("Detected %d CUDA Capable device(s)\n", deviceCount);
    }

    int dev = 0, driverVersion = 0, runtimeVersion = 0;
    CHECK(cudaSetDevice(dev));
    cudaDeviceProp deviceProp;
    CHECK(cudaGetDeviceProperties(&deviceProp, dev));
    printf("Device %d: \"%s\"\n", dev, deviceProp.name);

    cudaDriverGetVersion(&driverVersion);
    cudaRuntimeGetVersion(&runtimeVersion);
    printf("  CUDA Driver Version / Runtime Version          %d.%d / %d.%d\n",
           driverVersion / 1000, (driverVersion % 100) / 10,
           runtimeVersion / 1000, (runtimeVersion % 100) / 10);
    printf("  CUDA Capability Major/Minor version number:    %d.%d\n",
           deviceProp.major, deviceProp.minor);
    printf("  Total amount of global memory:                 %.2f GBytes (%llu "
           "bytes)\n", (float)deviceProp.totalGlobalMem / pow(1024.0, 3),
           (unsigned long long)deviceProp.totalGlobalMem);
    printf("  GPU Clock rate:                                %.0f MHz (%0.2f "
           "GHz)\n", deviceProp.clockRate * 1e-3f,
           deviceProp.clockRate * 1e-6f);
    printf("  Memory Clock rate:                             %.0f Mhz\n",
           deviceProp.memoryClockRate * 1e-3f);
    printf("  Memory Bus Width:                              %d-bit\n",
           deviceProp.memoryBusWidth);

    if (deviceProp.l2CacheSize)
    {
        printf("  L2 Cache Size:                                 %d bytes\n",
               deviceProp.l2CacheSize);
    }

    printf("  Max Texture Dimension Size (x,y,z)             1D=(%d), "
           "2D=(%d,%d), 3D=(%d,%d,%d)\n", deviceProp.maxTexture1D,
           deviceProp.maxTexture2D[0], deviceProp.maxTexture2D[1],
           deviceProp.maxTexture3D[0], deviceProp.maxTexture3D[1],
           deviceProp.maxTexture3D[2]);
    printf("  Max Layered Texture Size (dim) x layers        1D=(%d) x %d, "
           "2D=(%d,%d) x %d\n", deviceProp.maxTexture1DLayered[0],
           deviceProp.maxTexture1DLayered[1], deviceProp.maxTexture2DLayered[0],
           deviceProp.maxTexture2DLayered[1],
           deviceProp.maxTexture2DLayered[2]);
    printf("  Total amount of constant memory:               %lu bytes\n",
           deviceProp.totalConstMem);
    printf("  Total amount of shared memory per block:       %lu bytes\n",
           deviceProp.sharedMemPerBlock);
    printf("  Total number of registers available per block: %d\n",
           deviceProp.regsPerBlock);
    printf("  Warp size:                                     %d\n",
           deviceProp.warpSize);
    printf("  Maximum number of threads per multiprocessor:  %d\n",
           deviceProp.maxThreadsPerMultiProcessor);
    printf("  Maximum number of threads per block:           %d\n",
           deviceProp.maxThreadsPerBlock);
    printf("  Maximum sizes of each dimension of a block:    %d x %d x %d\n",
           deviceProp.maxThreadsDim[0],
           deviceProp.maxThreadsDim[1],
           deviceProp.maxThreadsDim[2]);
    printf("  Maximum sizes of each dimension of a grid:     %d x %d x %d\n",
           deviceProp.maxGridSize[0],
           deviceProp.maxGridSize[1],
           deviceProp.maxGridSize[2]);
    printf("  Maximum memory pitch:                          %lu bytes\n",
           deviceProp.memPitch);

    exit(EXIT_SUCCESS);
}


/tmp/tmpnj7cq7n6/fd0cc78d-12d4-4535-bb6e-d86205b99720.out Starting...
Detected 1 CUDA Capable device(s)
Device 0: "Tesla P100-PCIE-16GB"
  CUDA Driver Version / Runtime Version          10.1 / 10.1
  CUDA Capability Major/Minor version number:    6.0
  Total amount of global memory:                 15.90 GBytes (17071734784 bytes)
  GPU Clock rate:                                1329 MHz (1.33 GHz)
  Memory Clock rate:                             715 Mhz
  Memory Bus Width:                              4096-bit
  L2 Cache Size:                                 4194304 bytes
  Max Texture Dimension Size (x,y,z)             1D=(131072), 2D=(131072,65536), 3D=(16384,16384,16384)
  Max Layered Texture Size (dim) x layers        1D=(32768) x 2048, 2D=(32768,32768) x 2048
  Total amount of constant memory:               65536 bytes
  Total amount of shared memory per block:       49152 bytes
  Total number of registers available per block: 65536
  Warp size:                                   

* 1024

In [25]:
!./bt2 1024

**********GPU info**********
Name: Tesla P100-PCIE-16GB
Compute capability: 6.0
Num SMs: 56
Max num threads per SM: 2048
Max num warps per SM: 64
GMEM: 17071734784 bytes
****************************

Input size: 16777217

Kernel 1
Grid size: 8193, block size: 1024
Kernel time = 1.440032 ms, post-kernel time = 0.020352 ms
CORRECT :)

Kernel 2
Grid size: 8193, block size: 1024
Kernel time = 1.264512 ms, post-kernel time = 0.018720 ms
CORRECT :)

Kernel 3
Grid size: 8193, block size: 1024
Kernel time = 0.584640 ms, post-kernel time = 0.018976 ms
CORRECT :)


* 512

In [26]:
!./bt2 512

**********GPU info**********
Name: Tesla P100-PCIE-16GB
Compute capability: 6.0
Num SMs: 56
Max num threads per SM: 2048
Max num warps per SM: 64
GMEM: 17071734784 bytes
****************************

Input size: 16777217

Kernel 1
Grid size: 16385, block size: 512
Kernel time = 1.225440 ms, post-kernel time = 0.041568 ms
CORRECT :)

Kernel 2
Grid size: 16385, block size: 512
Kernel time = 1.101728 ms, post-kernel time = 0.038592 ms
CORRECT :)

Kernel 3
Grid size: 16385, block size: 512
Kernel time = 0.551616 ms, post-kernel time = 0.041472 ms
CORRECT :)


* 256

In [27]:
!./bt2 256

**********GPU info**********
Name: Tesla P100-PCIE-16GB
Compute capability: 6.0
Num SMs: 56
Max num threads per SM: 2048
Max num warps per SM: 64
GMEM: 17071734784 bytes
****************************

Input size: 16777217

Kernel 1
Grid size: 32769, block size: 256
Kernel time = 1.084576 ms, post-kernel time = 0.083904 ms
CORRECT :)

Kernel 2
Grid size: 32769, block size: 256
Kernel time = 0.976128 ms, post-kernel time = 0.165664 ms
CORRECT :)

Kernel 3
Grid size: 32769, block size: 256
Kernel time = 0.505920 ms, post-kernel time = 0.096064 ms
CORRECT :)


* 128

In [28]:
!./bt2 128

**********GPU info**********
Name: Tesla P100-PCIE-16GB
Compute capability: 6.0
Num SMs: 56
Max num threads per SM: 2048
Max num warps per SM: 64
GMEM: 17071734784 bytes
****************************

Input size: 16777217

Kernel 1
Grid size: 65537, block size: 128
Kernel time = 0.977472 ms, post-kernel time = 0.165504 ms
CORRECT :)

Kernel 2
Grid size: 65537, block size: 128
Kernel time = 0.886144 ms, post-kernel time = 0.166240 ms
CORRECT :)

Kernel 3
Grid size: 65537, block size: 128
Kernel time = 0.455392 ms, post-kernel time = 0.173696 ms
CORRECT :)


### Giải thích :

Xảy ra lỗi như vậy là do các thread của mỗi block được phân vào các wrap (32 thread) khác nhau.
Trong GPU thì mỗi thread trong một wrap thực thi bất đồng bộ sử dụng chung một thanh nhớ register (bộ nhớ tạm trên cùng một wrap) nên có thể với kích thước filter lớn hơn 32 thì quá trình tính toán song song sẽ dấn đến một thread đã xử lý xong và cập nhật kết quả mới vào bộ nhớ toàn cục nhưng thread khác đang xử lý sẽ lấy kết quả pixels từ bộ nhớ tạm trên thanh ghi.

In [29]:
!nvcc bt2.cu -o btdemo1 -Xptxas="-v"


ptxas info    : 0 bytes gmem
ptxas info    : Compiling entry function '_Z17reduceBlksKernel3PiiS_' for 'sm_30'
ptxas info    : Function properties for _Z17reduceBlksKernel3PiiS_
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 12 registers, 344 bytes cmem[0]
ptxas info    : Compiling entry function '_Z17reduceBlksKernel2PiiS_' for 'sm_30'
ptxas info    : Function properties for _Z17reduceBlksKernel2PiiS_
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 13 registers, 344 bytes cmem[0]
ptxas info    : Compiling entry function '_Z17reduceBlksKernel1PiiS_' for 'sm_30'
ptxas info    : Function properties for _Z17reduceBlksKernel1PiiS_
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 14 registers, 344 bytes cmem[0]


In [32]:
!nvcc bt2.cu -o btdemo2 --ptxas-options=-v

ptxas info    : 0 bytes gmem
ptxas info    : Compiling entry function '_Z17reduceBlksKernel3PiiS_' for 'sm_30'
ptxas info    : Function properties for _Z17reduceBlksKernel3PiiS_
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 12 registers, 344 bytes cmem[0]
ptxas info    : Compiling entry function '_Z17reduceBlksKernel2PiiS_' for 'sm_30'
ptxas info    : Function properties for _Z17reduceBlksKernel2PiiS_
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 13 registers, 344 bytes cmem[0]
ptxas info    : Compiling entry function '_Z17reduceBlksKernel1PiiS_' for 'sm_30'
ptxas info    : Function properties for _Z17reduceBlksKernel1PiiS_
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 14 registers, 344 bytes cmem[0]
